In [ ]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
# Download a Postgres driver that will allow Spark to interact with Postgres
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-05-03 18:45:54--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2020-05-03 18:45:54 (8.66 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [ ]:
# Start a Spark session with an additional option that adds the driver to Spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
# Import struct fields that we can use
from pyspark.sql.types import StructField, StringType, IntegerType, DateType, StructType

In [ ]:
# Next we need to create the list of struct fields
schema = [StructField("marketplace", StringType(), True), StructField("customer_id", IntegerType(), True),StructField("review_id", StringType(), True),StructField("product_id", StringType(), True),StructField("product_parent", IntegerType(), True),StructField("product_title", StringType(), True),StructField("product_category", StringType(), True),StructField("star_rating", IntegerType(), True),StructField("helpful_votes", IntegerType(), True),StructField("total_votes", IntegerType(), True),StructField("vine", StringType(), True),StructField("verified_purchase", StringType(), True),StructField("review_headline", StringType(), True),StructField("review_body", StringType(), True),StructField("review_date", DateType(), True)]
schema

[StructField(marketplace,StringType,true),
 StructField(customer_id,IntegerType,true),
 StructField(review_id,StringType,true),
 StructField(product_id,StringType,true),
 StructField(product_parent,IntegerType,true),
 StructField(product_title,StringType,true),
 StructField(product_category,StringType,true),
 StructField(star_rating,IntegerType,true),
 StructField(helpful_votes,IntegerType,true),
 StructField(total_votes,IntegerType,true),
 StructField(vine,StringType,true),
 StructField(verified_purchase,StringType,true),
 StructField(review_headline,StringType,true),
 StructField(review_body,StringType,true),
 StructField(review_date,DateType,true)]

In [ ]:
# Pass in our fields
final = StructType(fields=schema)
final

StructType(List(StructField(marketplace,StringType,true),StructField(customer_id,IntegerType,true),StructField(review_id,StringType,true),StructField(product_id,StringType,true),StructField(product_parent,IntegerType,true),StructField(product_title,StringType,true),StructField(product_category,StringType,true),StructField(star_rating,IntegerType,true),StructField(helpful_votes,IntegerType,true),StructField(total_votes,IntegerType,true),StructField(vine,StringType,true),StructField(verified_purchase,StringType,true),StructField(review_headline,StringType,true),StructField(review_body,StringType,true),StructField(review_date,DateType,true)))

In [ ]:
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Electronics_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
electronics_mod_data_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Electronics_v1_00.tsv.gz"), schema=final, sep="\t", header=True)

electronics_mod_data_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41409413|R2MTG1GCZLR2DK|B00428R89M|     112201306|yoomall 5M Antenn...|     Electronics|          5|            0|          0|   N|                Y|          Five Stars|       As described.| 2015-08-31|
|         US|   49668221|R2HBOEM8LE9928|B000068O48|     734576678|Hosa GPM-103 3.5m...|     Electronics|          5|    

In [ ]:
electronics_mod_data_df.describe()

DataFrame[summary: string, marketplace: string, customer_id: string, review_id: string, product_id: string, product_parent: string, product_title: string, product_category: string, star_rating: string, helpful_votes: string, total_votes: string, vine: string, verified_purchase: string, review_headline: string, review_body: string]

In [ ]:
electronics_mod_data_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



In [ ]:
# Count the number of records (rows) in the dataset
electronics_mod_data_df.count()

3093869

In [ ]:
# Drop any rows with null or “not a number” (NaN) values
electronics_clean_data_df = electronics_mod_data_df.dropna()
electronics_clean_data_df.count()

3093660

In [ ]:
electronics_clean_data_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   41409413|R2MTG1GCZLR2DK|B00428R89M|     112201306|yoomall 5M Antenn...|     Electronics|          5|            0|          0|   N|                Y|          Five Stars|       As described.|2015-08-31 00:00:00|
|         US|   49668221|R2HBOEM8LE9928|B000068O48|     734576678|Hosa GPM-103 3.5m...| 

In [ ]:
# Create user dataframe to match "review_id_table" table
cleaned_review_df = electronics_clean_data_df.select(["review_id","customer_id","product_id","product_parent","review_date"])
cleaned_review_df.show(10)

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
|R2MTG1GCZLR2DK|   41409413|B00428R89M|     112201306|2015-08-31 00:00:00|
|R2HBOEM8LE9928|   49668221|B000068O48|     734576678|2015-08-31 00:00:00|
|R1P4RW1R9FDPEE|   12338275|B000GGKOG8|     614448099|2015-08-31 00:00:00|
|R1EBPM82ENI67M|   38487968|B000NU4OTA|      72265257|2015-08-31 00:00:00|
|R372S58V6D11AT|   23732619|B00JOQIO6S|     308169188|2015-08-31 00:00:00|
|R1A4514XOYI1PD|   21257820|B008NCD2LG|     976385982|2015-08-31 00:00:00|
|R20D9EHB7N20V6|    3084991|B00007FGUF|     670878953|2015-08-31 00:00:00|
|R1WUTD8MVSROJU|    8153674|B00M9V2RMM|     508452933|2015-08-31 00:00:00|
|R1QCYLT25812DM|   52246189|B00J3O9DYI|     766372886|2015-08-31 00:00:00|
| R904DQPBCEM7A|   41463864|B00NS1A0E4|     458130381|2015-08-31 00:00:00|
+--------------+---------

In [ ]:
cleaned_review_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: timestamp (nullable = true)



In [ ]:
# Create user dataframe to match "products" table
cleaned_products_df = electronics_clean_data_df["product_id","product_title"].distinct()
cleaned_products_df.show(10)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00JL2498C|Jensen JTA-410-BL...|
|B00OUKG3NE|Sennheiser Androi...|
|B0083H9R6A|Electrified ELPLP...|
|B0097BEE9Q|Apple iPod nano 1...|
|B0037Z4MCM|Pyle Marine Sport...|
|B00P0BQKBC|Water & Wood Lock...|
|B00TP1C1UC|AmazonBasics 6-Ou...|
|B005AIF0X4|iHip MVF1030WO Ma...|
|B009JJKZSK|Click 'N Dig! Key...|
|B00KXDS4VQ|enKo products HDM...|
+----------+--------------------+
only showing top 10 rows



In [ ]:
# Count the rows
cleaned_products_df.count()

185842

In [ ]:
cleaned_products_df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [ ]:
# Group the reviews by customer id and count them
customerid_count_df = electronics_clean_data_df.groupBy(electronics_clean_data_df["customer_id"]).count()
customerid_count_df.show(10)

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   42560427|    6|
|   28871066|    1|
|    6711334|    2|
|   46393060|    1|
|   48101124|    1|
|   38209321|    1|
|   47108763|    4|
|   43252205|    1|
|   41283385|    1|
|   46909180|    5|
+-----------+-----+
only showing top 10 rows



In [ ]:
customerid_count_df.count()

2154211

In [ ]:
customerid_count_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- count: long (nullable = false)



In [ ]:
# Create user dataframe to match "customers" table
customers_df = customerid_count_df.withColumnRenamed('count','customer_count')
customers_df.show(10)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   42560427|             6|
|   28871066|             1|
|    6711334|             2|
|   46393060|             1|
|   48101124|             1|
|   38209321|             1|
|   47108763|             4|
|   43252205|             1|
|   41283385|             1|
|   46909180|             5|
+-----------+--------------+
only showing top 10 rows



In [ ]:
# Create user dataframe to match "vine_table" table
cleaned_vine_df = electronics_clean_data_df.select(["review_id","star_rating","helpful_votes","total_votes","vine"])
cleaned_vine_df.show(10)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R2MTG1GCZLR2DK|          5|            0|          0|   N|
|R2HBOEM8LE9928|          5|            0|          0|   N|
|R1P4RW1R9FDPEE|          5|            1|          1|   N|
|R1EBPM82ENI67M|          1|            0|          0|   N|
|R372S58V6D11AT|          5|            1|          1|   N|
|R1A4514XOYI1PD|          5|            1|          1|   N|
|R20D9EHB7N20V6|          5|            0|          0|   N|
|R1WUTD8MVSROJU|          5|            0|          0|   N|
|R1QCYLT25812DM|          4|            0|          0|   N|
| R904DQPBCEM7A|          4|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



In [ ]:
cleaned_vine_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://archanadatabase.cb90x2qdg1xw.us-west-1.rds.amazonaws.com:5432/Amazon_reviews"
config = {"user":"postgres", 
          "password": "#Ar1707#", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to "review_id_table" table in RDS
cleaned_review_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write DataFrame to "products" table in RDS
cleaned_products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write DataFrame to "customers" table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Write DataFrame to "vine_table" table in RDS
cleaned_vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)